In [1]:
import spotipy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re

load_dotenv('../secrets/.env', override=True)

%run useful_functions.ipynb
%run spotify_scraping.ipynb

scope = "playlist-modify-private"

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

my_id = spotify.current_user()['id']

#https://accounts.spotify.com/authorize
# ?client_id=13e600b84519420f9289a1250b1894da
# &response_type=code&
# redirect_uri=http%3A%2F%2Flocalhost%3A6000%2Fcallback%2F
# &scope=playlist-modify-private

In [2]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

master_df = (
broadcast_playlists
.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()

In [3]:
rhead = artists[artists['Artist Name'] == 'Queen']['Artist ID'][0]
rh_tracks = tracks[tracks["Artist ID's"].apply(lambda x: True if (rhead in x) else False)]

In [4]:
rh_tracks

,Disc Number,Track Duration (ms),Is Track Explicit,Track External ID's,Track External URL's,Track href,Track ID,Is Track Local,Is Track Playable,Track Name,...,Track Number,Track Type,Track URI,Original Search Artist,Original Search Track,Cleaned Search Artist,Cleaned Search Track,Search Query,Album ID,Artist ID's
0,1,354320,False,{'isrc': 'GBUM71029604'},{'spotify': 'https://open.spotify.com/track/4u...,https://api.spotify.com/v1/tracks/4u7EnebtmKWz...,4u7EnebtmKWzUH433cf5Qv,False,True,Bohemian Rhapsody - Remastered 2011,...,11,track,spotify:track:4u7EnebtmKWzUH433cf5Qv,Queen,Bohemian Rhapsody,Queen,Bohemian Rhapsody,artist:Queen track:Bohemian Rhapsody,1GbtB4zTqAsyfZEsm1RZfx,[1dfeR4HaWDbWqFHLkxsg1d]


In [5]:
#spotify.audio_analysis("4u7EnebtmKWzUH433cf5Qv")
#spotify.audio_features("4u7EnebtmKWzUH433cf5Qv")
#spotify.recommendation_genre_seeds()

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1571899663,
  'analysis_time': 9.64977,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 7812756,
  'duration': 354.32,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.1795,
  'start_of_fade_out': 334.1061,
  'loudness': -9.928,
  'tempo': 71.105,
  'tempo_confidence': 0.058,
  'time_signature': 4,
  'time_signature_confidence': 0.899,
  'key': 0,
  'key_confidence': 0.085,
  'mode': 0,
  'mode_confidence': 0.23,
  'codestring': 'eJxVm4eVJDcMBVOZEOhN_omp6nNWtys9nY6YbhoQ5sP0nXes2lb9lE8dcx1Gn7rKZ445St3301v53NJvm3v3zyrnM9usu-zBG6WNzxinzXHq-NQ626fVwv_uOMzHI-fufVY7_dOaQxbZdTNnP04wRlt3nc8ohTU7g8L4M1y137r6LbN_xtrtM86q6871meXwwq2n3dXqZ84yPqzg_hq_bnc_D4dpe3_WZk8M-9n73s-6tbF0b3tyvs9mis-ud9092vnswzpr93OOM596G2esbfZ--_gcT3_u6Lw8GW42Ps-8-xyevoNf763M

In [19]:
test_playlist = spotify.user_playlist_create(my_id, "test_playlist", public=False, collaborative=False, description='Test Playlist')
playlist_id = test_playlist['id']
tracks_to_add = list(rh_tracks['Track ID'])
i = 0
while i < len(tracks_to_add) // 50 + 1:
    temp_tracks_to_add = tracks_to_add[50*i:50*(i+1)]
    spotify.user_playlist_add_tracks(my_id, playlist_id, temp_tracks_to_add, position=50*i)
    i += 1

NameError: name 'my_id' is not defined

In [14]:
user_playlist_remove_all_occurrences_of_tracks(user, playlist_id, tracks, snapshot_id=None)

In [20]:
spotify.playlist_add_items(playlist_id, [temp_tracks_to_add[0]], position=0)

NameError: name 'playlist_id' is not defined

In [32]:
tracks_to_add[50:100]


['1QG6fYUoExhAMjzVi8EdIQ',
 '77rF32ZnoNMqQlxWjwTBCn',
 '20Dw7Jar5hUbwX5FwHdQoG',
 '6yLkSsbvHSIzjBBVoIxdM6',
 '0ByROqFlBxTQeX4XlvE7Gk',
 '4rXfbIpiDnGxZYIflm8LSQ']